In [86]:
#Python version
!python --version

Python 3.8.5


In [87]:
import numpy as np
import cv2
import os
import glob
import random
import shutil

from scipy.spatial import distance
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras import Sequential, models, activations
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPool2D
from tensorflow.keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import seaborn as sns

In [88]:
mask_data = "Facemask-Dataset/Mask/Mask/" 
no_mask_data = "Facemask-Dataset/No Mask/No Mask/"

total_mask_images = os.listdir(mask_data)
print("no of mask images:: {}".format(len(total_mask_images)))
total_nonmask_images = os.listdir(no_mask_data)
print("no of non-mask images:: {}".format(len(total_nonmask_images)))

no of mask images:: 208
no of non-mask images:: 131


In [89]:
'''
os.makedirs('./train/mask')
os.makedirs('./train/no mask')
os.makedirs('./test/mask')
os.makedirs('./test/no mask')
'''

"\nos.makedirs('./train/mask')\nos.makedirs('./train/no mask')\nos.makedirs('./test/mask')\nos.makedirs('./test/no mask')\n"

In [90]:
trainNum = 100
testNum = 30

for images in random.sample(total_mask_images,trainNum):
    shutil.copy(mask_data+images, './train/mask')
for images in random.sample(total_mask_images,testNum):
    shutil.copy(mask_data+images, './test/mask')
for images in random.sample(total_nonmask_images,trainNum):
    shutil.copy(no_mask_data+images, './train/no mask')
for images in random.sample(total_nonmask_images,testNum):
    shutil.copy(no_mask_data+images, './test/no mask')

In [91]:
train_batch = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.2).\
            flow_from_directory('./train', target_size=(224,224), batch_size=32, class_mode = 'categorical')
test_batch = ImageDataGenerator(
    rescale=1./255).\
            flow_from_directory('./test', target_size = (224,224), batch_size=32, class_mode='categorical')

Found 332 images belonging to 2 classes.
Found 233 images belonging to 2 classes.


In [92]:
train_batch.class_indices

{'mask': 0, 'no mask': 1}

In [93]:
class_mask = ['mask', 'no mask']

In [94]:
model = Sequential()
model.add(Conv2D(8,(3, 3), padding="same", input_shape=(32, 32, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(16,(3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(32,(3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(3))
model.add(Activation("softmax"))

print("[INFO training network...]")
opt = Adam(lr=1e-3, decay=1e-3/50)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
H = model.fit(trainX, trainY, validation_data=(testX, testY), epochs=50, batch_size=32)

print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=lb.classes_))

NameError: name 'Activation' is not defined